In [2]:
import pandas as pd

Unemployment Data by county (https://www.bls.gov/lau/#tables)

In [15]:
list_ = []
for x in range(2010,2019):
    df = pd.read_fwf(f'https://www.bls.gov/lau/laucnty{str(x)[-2:]}.txt', skiprows = 5)
    list_.append(df)
all_data = pd.concat(list_)
all_data = all_data.dropna()
all_data = all_data.rename(columns = {
    'Unnamed: 3' : 'county_name',
    'Unnamed: 4' : 'year',
    'Unnamed: 8' : 'unemployment_rate'
})

In [3]:
all_data.year = all_data.year.astype(int)
unemployment_data = all_data[['year','county_name','unemployment_rate']]
# unemployment_data[['county','state']] = unemployment_data.county_name.str.split(',', expand = True)
# unemployment_data = unemployment_data[['year', 'county', 'state', 'unemployment_rate']]

In [5]:
zip_county_key = pd.read_csv('zip_county_key.csv')
zip_county_key = zip_county_key.rename(columns = {
    'COUNTYNAME' : 'county',
    'STATE' : 'state'
})
zip_county_key['county_name'] = zip_county_key['county'] + ", " + zip_county_key['state']

In [6]:
unemployment_data = pd.merge(unemployment_data,
             zip_county_key,
             how = 'left',
             on = ['county_name'])
unemployment_data = unemployment_data[['year', 'ZIP', 'unemployment_rate']]

In [7]:
unemployment_data = unemployment_data.rename(columns = {'year': 'Year', 'ZIP_x' : 'ZIP'})

In [9]:
unemployment_data.to_csv('unemployment_data.csv')

Income Data by County (https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-zip-code-data-soi)

In [2]:
# df = pd.read_csv('https://www.irs.gov/pub/irs-soi/16zpallagi.csv')[['zipcode', 'agi_stub', 'N1']]

In [3]:
list_ = []
for x in range(2011,2017):
    df = pd.read_csv(f'https://www.irs.gov/pub/irs-soi/{str(x)[-2:]}zpallagi.csv')
    df.columns = map(str.lower, df.columns)
    df = df[['zipcode', 'agi_stub', 'n1']]
    list_.append(df)
    df['year'] = x
income_master = pd.concat(list_)

In [4]:
income_master.to_csv('income_placeholder_v3.csv')

In [5]:
income = []
for index, row in income_master.iterrows():
    if row['agi_stub'] == 1:
        income.append(12500)
    elif row['agi_stub'] == 2:
        income.append(37500)
    elif row['agi_stub'] == 3:
        income.append(62500)
    elif row['agi_stub'] == 4:
        income.append(87500)
    elif row['agi_stub'] == 5:
        income.append(150000)
    elif row['agi_stub'] == 6:
        income.append(500000)
        
income_master['income'] = income

In [6]:
def wavg(val_col_name, wt_col_name):
    def inner(group):
        return (group[val_col_name] * group[wt_col_name]).sum() / group[wt_col_name].sum()
    inner.__name__ = 'wtd_avg'
    return inner

income_master_test= pd.DataFrame(income_master.groupby(['zipcode', 'year']).apply(wavg('income', 'n1')))

In [7]:
# income_master_test = income_master_test['WAVG']

In [8]:
income_master_test.to_csv('test_income_v3.csv', index=True)

In [9]:
income_master_test

0
zipcode year              
0       2011  61533.618577
        2012  64201.048160
        2013  65779.966167
        2014  68656.931018
        2015  70783.890594
        2016  71891.960247
1001    2011  48135.470992
        2012  57332.004556
        2013  59923.120729
        2014  61416.103604
        2015  62569.832402
        2016  63237.750557
1002    2011  74861.083792
        2012  83853.065539
        2013  88153.082550
        2014  91390.041494
        2015  93872.802482
        2016  95943.097998
1003    2016  15441.176471
1005    2011  59671.740118
        2012  60594.170404
        2013  64125.560538
        2014  66831.140351
        2015  67500.000000
        2016  65400.843882
1007    2011  71242.214533
        2012  73770.491803
        2013  76335.616438
        2014  79228.687415
        2015  82793.724967
...                    ...
99827   2016  58169.291339
99829   2016  44871.794872
99830   2016  37500.000000
99833   2015  68862.275449
        2016  68222.891566
99835   2011  60165.931527
        2012  62231.182796
        2013  66086.956522
        2014  67526.881720
        2015  70021.881838
        2016  67877.252252
99840   2016  54707.792208
99901   2011  60070.685102
        2012  64178.515008
        2013  65673.676012
        2014  68376.572327
        2015  71767.912773
        2016  69246.031746
99919   2016  52403.846154
99921   2016  53915.662651
99922   2016  32954.545455
99925   2016  50872.093023
99926   2016  44545.454545
99929   2016  50817.757009
99999   2011  56143.227806
        2012  58621.104612
        2013  58977.805787
        2014  64206.857005
        2015  66299.926219
        2016  70028.468239

[168443 rows x 1 columns]